In [1]:
import tensorflow as tf 
import tensorflow.keras as keras
tf.random.set_seed(2345)

# 因为默认tf会吧全部显存都占用光, 所以在进行卷积计算的时候回报错提示无法初始化cuDNN, 所以这里我们需要手动的将tf设置成按需占用内存
# 这样就避免了卷积计算错误的问题
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [2]:
(data_train,label_train),(data_test,label_test) = keras.datasets.cifar100.load_data()
label_train = tf.squeeze(label_train,axis=1)
label_test = tf.squeeze(label_test,axis=1)


In [3]:
data_train.shape,label_train.shape 

((50000, 32, 32, 3), TensorShape([50000]))

In [4]:
# 定义五个卷积单元, 我们使用每个卷积单元是2个卷积层串联加一个maxpooling层
conv_layers = [
    # unit 1
    keras.layers.Conv2D(64,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),# 卷积层也可以使用激活函数
    keras.layers.Conv2D(64,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='SAME'),
    
    # unit 2
    keras.layers.Conv2D(128,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),# 卷积层也可以使用激活函数
    keras.layers.Conv2D(128,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='SAME'),
    
    # unit 3
    keras.layers.Conv2D(256,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),# 卷积层也可以使用激活函数
    keras.layers.Conv2D(256,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='SAME'),
    
    # unit 4
    keras.layers.Conv2D(512,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),# 卷积层也可以使用激活函数
    keras.layers.Conv2D(512,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='SAME'),
    
    # unit 5
    keras.layers.Conv2D(512,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),# 卷积层也可以使用激活函数
    keras.layers.Conv2D(512,kernel_size=[3,3],strides=1,padding='SAME',activation=tf.nn.relu),
    keras.layers.MaxPool2D(pool_size=[2,2],strides=2,padding='SAME'),
]
# 这样就得到了一个10层的卷积神经网络


In [5]:
def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)/255. 
    y = tf.cast(y,dtype=tf.int32)
    return x,y

In [13]:
def main():
    train_db = tf.data.Dataset.from_tensor_slices((data_train,label_train))
    train_db = train_db.shuffle(10000).map(preprocess).batch(256)
    
    test_db = tf.data.Dataset.from_tensor_slices((data_test,label_test))
    test_db = test_db.shuffle(10000).map(preprocess).batch(256)
    
    conv_net = keras.Sequential(conv_layers)
    
#     x = tf.random.normal([4,32,32,3])
#     print(conv_net(x).shape)
    fc_net = keras.Sequential([
        keras.layers.Dense(256,activation=tf.nn.relu),
        keras.layers.Dense(128,activation=tf.nn.relu),
        keras.layers.Dense(100,activation=tf.nn.relu)
    ])
    conv_net.build(input_shape=[None,32,32,3])
    fc_net.build(input_shape=[None,512]) 
    variables =conv_net.trainable_variables + fc_net.trainable_variables
    optimizer = keras.optimizers.Adam(learning_rate=1e-4)
    
    for epoch in range(50):
        for step,(x,y) in enumerate(train_db):
            with tf.GradientTape() as tape:
                out = conv_net(x)
                # [b,32,32,3]转变为了[b,1,1,512]的矩阵
                out = tf.reshape(out,[-1,512])
                
                # 再传入到全连接层中
                # [b,512]转变为了[b,100]
                logits = fc_net(out)
                y_onehot = tf.one_hot(y,depth=100)
                loss = tf.losses.categorical_crossentropy(y_onehot,logits,from_logits=True)
                loss = tf.reduce_mean(loss)
                
            grads = tape.gradient(loss,variables)
            optimizer.apply_gradients(zip(grads,variables))
            if step%100 == 0:
                print(epoch,step,'loss: ',loss.numpy())
        
        total_num = 0
        total_correct = 0
        for (x,y) in test_db:
            out = conv_net(x)
            out = tf.reshape(out,[-1,512])
            logits = fc_net(out)
            prob = tf.nn.softmax(logits,axis=1)
            pred = tf.argmax(prob,axis=1)
            pred = tf.cast(pred,dtype=tf.int32)
            
            correct = tf.cast(tf.equal(pred,y),dtype=tf.int32)
            correct = tf.reduce_sum(correct)
            total_correct+= int(correct)
            total_num += x.shape[0]
        acc = total_correct/total_num
        print(epoch,'Accuracy: ',acc)

In [14]:
main()

0 0 loss:  5.5419426
0 100 loss:  4.174409
0 Accuracy:  0.1328
1 0 loss:  4.000986
1 100 loss:  3.991229
1 Accuracy:  0.1728
2 0 loss:  3.9446347
2 100 loss:  3.71436
2 Accuracy:  0.1886
3 0 loss:  3.7404895
3 100 loss:  3.7387748
3 Accuracy:  0.2101
4 0 loss:  3.5604115
4 100 loss:  3.4384232
4 Accuracy:  0.2174
5 0 loss:  3.540745
5 100 loss:  3.3633566
5 Accuracy:  0.2353
6 0 loss:  3.3984401
6 100 loss:  3.422825
6 Accuracy:  0.2529
7 0 loss:  3.2521577
7 100 loss:  3.2374184
7 Accuracy:  0.2631
8 0 loss:  3.365357
8 100 loss:  3.1660628
8 Accuracy:  0.2824
9 0 loss:  3.0835934
9 100 loss:  3.0234213
9 Accuracy:  0.2893
10 0 loss:  3.1217113
10 100 loss:  3.045132
10 Accuracy:  0.2942
11 0 loss:  3.189695
11 100 loss:  2.9459574
11 Accuracy:  0.2991
12 0 loss:  2.6475303
12 100 loss:  2.7945535
12 Accuracy:  0.314
13 0 loss:  3.0108073
13 100 loss:  2.930768
13 Accuracy:  0.3218
14 0 loss:  2.8861532
14 100 loss:  2.6064332
14 Accuracy:  0.3222
15 0 loss:  2.730177
15 100 loss:  2.